# User API

Full documentation of the Rescale APi is available at [https://engineering.rescale.com/api-docs](https://engineering.rescale.com/api-docs). This document builds a story that covers the entire User API surface.

## Starting the notebook

The only prerequisite to running this tutorial locally is a working installation of Python 3.8+. We're going to work within a virtual environment so that our dependencies are installed locally without affecting the global installation.

```
$ python -m venv .venv
$ . .venv/bin/activate
$ pip install -r requirements.txt
$ jupyter notebook README.ipynb
```

Jupyter Notebook cells need to executed in order as subsequent cells may depend on imports and variables defined in previous cells.

### Python and JSON

> TODO: Note on how Python dictionaries are 1-1 mapped to JSON.

## Authorization

The Rescale API authorizes requests using `API Key`. This key is unique per User, per Workspace. A give user may have multiple `API Keys` active - one per Workspace.

To generate an `API Key` go to the API section of the User Profile.

![](README.images/user_profile_apikey.png)

> NOTE: Remember that your `API Key` is a secret. Never store it in a code repository. Never share it with your colleagues.

Rescale tools like Rescale CLI look for `apiconfig` authorization configuration file in users home directory.

```
$HOME/.config/rescale/apiconfig             # Linux
%USERPROFILE%\config\rescale\apiconfig      # Windows
```

We will use this convention to store our credentials. Create the `apiconfig` text file in the aforementioned location and fill it with the following lines

```
[default]
apibaseurl = https://eu.rescale.com
apikey = 79a49b3132335a44742e86c9126e5cfaa1ea2489
```

If you need to execute your script on several platforms you can define alternative profiles (fo example `[us]`). The [config.py](config.py) module is provided for your convenience. You can copy it next to your scripts and use it as a standard way to retrieve credentials. Let's use it.

In [ ]:
import config

api_url, api_key = config.get_profile()


We now have our `apikey` and a base URL for our API calls. Let's make the first call to get our user details. To make REST API calls we will use the [`requests`](https://requests.readthedocs.io/en/latest/) module. It is listed as a dependency in the [`requirements.txt`](requirements.txt) file.

In [ ]:
import requests

results = requests.get(f"{api_url}/api/v2/users/me/")
if results.status_code != 200:
    print(results.status_code)


We got status code `401 Unauthorized` which suggests that we're missing our credentials. Let's define `headers` that we will use to authorize future API calls. Instead of checking for result code, let's use a function that raises exception for all statuses that signify lack of success. Finally, we pretty-print the response JSON document.

In [ ]:
headers = {"Authorization": f"Token {api_key}"}
results = requests.get(f"{api_url}/api/v2/users/me/", headers=headers)
results.raise_for_status()

from pprint import pprint

pprint(results.json())


We're now ready to proceed and build up job submission.

## Coretypes and Analyses


The job definition requires us to specify software to be attached to a cluster running our job and the hardware (coretype and cores count). Let's [get the list of coretypes](https://engineering.rescale.com/api-docs/#coretypes).

In [ ]:
coretypes_res = requests.get(f"{api_url}/api/v2/coretypes/", headers=headers)
coretypes_res.raise_for_status()

# Display the total count of coretypes and the amount present in the response
print(
    f"Count: {coretypes_res.json()['count']}; Length: {len(coretypes_res.json()['results'])}"
)

# Display refernece to the next page
print(f"Next page: {coretypes_res.json()['next']}")

# Display a coretype object and its properties
pprint(coretypes_res.json()["results"][0])


The first thing to note is that the response returned total count of coretypes that is larger the the total count of coretype objects in the `results` list. This will happen often for endpoints that potentially return large amounts of data. In such situations results are paged and each page contains a reference to the `next` page.

Since paging is common, let's define a function that will loop through all the pages and aggregate objects returned in the `results` list of each page.

In [ ]:
def get_all_result_pages(url, headers={}, params={}):
    results = []

    res = requests.get(url, headers=headers, params=params)
    res.raise_for_status()

    results.extend(res.json()["results"])

    while res.json()["next"] != None:
        res = requests.get(res.json()["next"], headers=headers)
        res.raise_for_status()
        results.extend(res.json()["results"])

    return results


coretypes = get_all_result_pages(f"{api_url}/api/v2/coretypes/", headers)

print(f"Length: {len(coretypes)}")


We're now confident that we fetched all coretypes. For our simple job we will need a general purpose coretype with a low corecount. Let's list coretype `code`s in the `general` category together with their `cores` counts and `processorInfo`.

In [ ]:
general_coretypes = {
    c['code']: {'cores': c['cores'], 'processorInfo': c['processorInfo']}
    for c in coretypes
    if "general" in c["categoryCodes"]
}

pprint(general_coretypes)

Now that we have a shortlist we need to decide which coretype to use. Since our test calculation does not have specific requirements, we will use the least expensive option. Let's [fetch coretype prices](https://engineering.rescale.com/api-docs/#list-all-compute-prices), filter prices for on-demand economy (ODE) an `linux` os, link them with general purpose coretypes and get the minium price.

In [ ]:
# Get prices
prices_res = requests.get(f"{api_url}/api/v2/billing/computeprices/", headers=headers)
prices_res.raise_for_status()

# Print sample pricing object
pprint(prices_res.json()[-1])

# Filter out active ODE linux based coretypes
linux_ode_coretypes = {
    c["coreType"]: float(c["value"]["amount"])
    for c in prices_res.json()
    if c["planType"].endswith("on-demand")
    and c["os"] == "linux"
    and c["isActive"]
    and c["coreType"] in general_coretypes
}

# Find the least expensive one
min_price_coretype = min(linux_ode_coretypes, key=linux_ode_coretypes.get)
print(f"Lowest price coretype: {min_price_coretype} at {linux_ode_coretypes[min_price_coretype]}")


Spot market prices are dynamic and therefore your cheapest coretype may change even during a day. Let's assume that `granite` is the coretype we want to use.

Now, that we know which coretype to use, we need to select the analysis. In order to build our job create request, we need to know analysis and version codes. Let's query API for the [list of available analyses](https://engineering.rescale.com/api-docs/#analyses) (software tiles).

In [ ]:
analyses = get_all_result_pages(f"{api_url}/api/v2/analyses/", headers)

versions_count = 0
for a in analyses:
    versions_count += len(a["versions"])

print(
    f"Total number of analyses: {len(analyses)}, Total number of versions: {versions_count}"
)
print(
    f"Sample analysis code: {analyses[0]['code']} and version code: {analyses[0]['versions'][0]['versionCode']}"
)

pprint(analyses[0])


If your workspace does not have software filters in place, you should see over 700 analyses and over 3200 unique versions. It may be cumbersome to find the code and version that you need. A more practical approach is to create a job using the web portal and then query Rescale API to get the JSON. We will do it in the next section.

## Jobs and Files

In order to fetch the definition of a previously saved Job, go to the portal and get a Job ID for a job similar to the one you'd like to create via the Rescale API.

![](README.images/webportal_jobid.png)

Once we captured the Job ID we can [get a specific Job](https://engineering.rescale.com/api-docs/#get-a-specific-job).

In [ ]:
# Get details of a job
job_id = "jNibFc"

results = requests.get(f"{api_url}/api/v2/jobs/{job_id}", headers=headers)
results.raise_for_status()

# Display the analysis section
pprint(results.json()["jobanalyses"][0]["analysis"])

# Display the entire job definition JSON
pprint(results.json())


Note that `analyses` property is a list. This represents a possibility of having multiple software tiles (analyses) attached to a single Job.

Rescale API also allows to [list all user jobs](https://engineering.rescale.com/api-docs/#list-all-jobs). Let's count how many jobs completed this year. Additional query parameters allow filtering jobs by:

* `state` - one of `(completed, not_completed)`
* `job_status` - one of `(PENDING, QUEUED, STARTED, VALIDATED, EXECUTING, COMPLETED, STOPPING, WAITING_FOR_CLUSTER, FORCE_STOP, WAITING_FOR_QUEUE)`

In [ ]:
jobs = get_all_result_pages(
    f"{api_url}/api/v2/jobs/", headers, params={"state": "completed"}
)

from datetime import datetime

this_year = datetime.today().year

# dateInserted does not conform with ISO format - a small replacement is needed
this_year_jobs = [
    j
    for j in jobs
    if datetime.fromisoformat(j["dateInserted"].replace("Z", "+00:00")).year
    == this_year
]
print(f"Total jobs: {len(jobs)}, Jobs in {this_year}: {len(this_year_jobs)}")


Let's follow by creating a new Job. We will use `miniconda` analysis to run a [Python script](job_inputs/calculate_pi.py) which estimates the value of π using the the [Leibniz’s formula](https://en.wikipedia.org/wiki/Leibniz_formula_for_%CF%80). The script requires an [input file](job_inputs/range.inp) which specifies the number of iterations. Estimated value is stored as text in `pi_estimate.res` and in a binary format in `pi_estimate.bin`.

> NOTE: If your Workspace does not have the `Miniconda` software enabled, you can use any analysis as all Rescale clusters have a Python interpreter available. CHECK: Do we need a non-licensed software here?

It may be the case, that your Rescale setup requires jobs to be submitted against a specific Project. Let's [list available projects](https://engineering.rescale.com/api-docs/#list-projects-available-to-your-user).

In [ ]:
projects = get_all_result_pages(f"{api_url}/api/v2/users/me/projects/", headers)

for p in projects:
    print(f"{p['id']}\t{p['name']}")


Now that we have all needed information about the hardware (`coreType: granite`) and software (`code: miniconda; version: 4.8.4`) we can specify our job definition payload and [create a Job](https://engineering.rescale.com/api-docs/#create-a-job). To keep things simple in this tutorial, we're not handling exceptions raised by our HTTP requests. Production code, should try to recover fro an exception or give end user information on how to proceed. Here, we expect an exception, so we will catch it and display an error, to demonstrate Rescale API returning useful error messages.

In [ ]:
# Replace with your projectId if required/desired
project_id = None

job_definition = {
    "name": "Rescale UserApi Reference Tutorial",
    "jobanalyses": [
        {
            "analysis": {"code": "miniconda", "version": "4.8.4"},
            "command": "python calculate_pi.py range.inp",
            "hardware": {"coreType": "granite", "coresPerSlot": 1},
        }
    ],
    "projectId": project_id,
}

job_save_res = requests.post(
    f"{api_url}/api/v2/jobs/", headers=headers, json=job_definition
)
try:
    job_save_res.raise_for_status()
except:
    pprint(job_save_res.json())


The issue is that the coretype we have selected, `granite`, does not support configuration with `1` core per slot. Let's fix it by updating the Job definition.

In [ ]:
job_definition["jobanalyses"][0]["hardware"]["coresPerSlot"] = 2

job_create_res = requests.post(
    f"{api_url}/api/v2/jobs/", headers=headers, json=job_definition
)
job_create_res.raise_for_status()

pprint(job_create_res.json())


The job was successfully created. We can check whether it is visible in the Rescale web portal.

![](README.images/webportal_newjob.png)

Job creation operation just saved the job, it has not yet been submitted. In order to [submit a job](https://engineering.rescale.com/api-docs/#submit-a-saved-job) we need to capture its ID and call the `submit` operation.

In [ ]:
job_id = job_create_res.json()["id"]

job_submit_res = requests.post(
    f"{api_url}/api/v2/jobs/{job_id}/submit/", headers=headers
)
job_submit_res.raise_for_status()

print(f"Status code: {job_submit_res.status_code}")


Status `200` signifies a success. If we go back to the web portal, we will see job statuses changing.

![](README.images/webportal_submittedjob.png)

We want to wait until the job finishes. Let us poll for [job statuses](https://engineering.rescale.com/api-docs/#list-job-status-history) programmatically. We will define an utility function that polls until a specific Job state is reached.

In [ ]:
def poll_for_job_status(job_id, status, interval=30):
    import time

    while True:
        job_statuses = get_all_result_pages(
            f"{api_url}/api/v2/jobs/{job_id}/statuses/", headers=headers
        )
        sorted_statuses = sorted(
            job_statuses,
            key=lambda s: datetime.fromisoformat(s["statusDate"]),
        )
        status_items = [s for s in sorted_statuses if s["status"] == status]
        print(" > ".join([s["status"] for s in sorted_statuses]))

        if len(status_items) > 0:
            return status_items[0]

        time.sleep(interval)


poll_for_job_status(job_id, "Completed")


Our job went through all the statuses and reached the desired terminal state, however, it seems that the calculation failed. Let's try to figure out why by [listing output files](https://engineering.rescale.com/api-docs/#list-job-output-files). We will search for files that have `output` string in their name.

In [ ]:
output_files = get_all_result_pages(
    f"{api_url}/api/v2/jobs/{job_id}/files/", headers, params={"search": "output"}
)

pprint(output_files)

The `process_output.log` file is created for all jobs and captures everything written to a console (standard output and error streams). Let's [fetch plaintext contents](https://engineering.rescale.com/api-docs/#get-plaintext-content-of-a-file) of this file.

In [ ]:
file_id = output_files[0]["id"]

file_contents_response = requests.get(
    f"{api_url}/api/v2/files/{file_id}/lines", headers=headers
)
file_contents_response.raise_for_status()

for line in file_contents_response.json()["lines"]:
    print(line, end="")

Ha! All is clear. We forgot to upload input files. Let's [upload files](https://engineering.rescale.com/api-docs/#upload-a-file) and capture their IDs. Since file upload are frequent operation we will encapsulate in a function.

In [ ]:
def file_upload(file_path):
    import os

    files = [
        (
            "file",
            (
                os.path.basename(file_path),
                open(file_path, "rb"),
                "application/octet-stream",
            ),
        )
    ]

    file_upload_res = requests.post(
        f"{api_url}/api/v2/files/contents/", headers=headers, files=files
    )
    file_upload_res.raise_for_status()

    return file_upload_res.json()["id"]


file1_id = file_upload("job_inputs/calculate_pi.py")
file2_id = file_upload("job_inputs/range.inp")

print(file1_id, file2_id)


Let's try to update our previous job definition by extending the job analysis specification with the `inputFiles` property, and resubmit the job.

In [ ]:
job_definition["jobanalyses"][0]["inputFiles"] = [{"id": file1_id}, {"id": file2_id}]
pprint(job_definition)

job_update_res = requests.patch(
    f"{api_url}/api/v2/jobs/{job_id}", headers=headers, json=job_definition
)
job_update_res.raise_for_status()
pprint(job_update_res.status_code)

job_submit_res = requests.post(
    f"{api_url}/api/v2/jobs/{job_id}/submit/", headers=headers
)
try:
    job_submit_res.raise_for_status()
except:
    print(job_submit_res.status_code)
    print(job_submit_res.json())


Now. Although the API responds with `200 OK` for `PATCH`, both of the above operations had no effect as changing or re-submitting a Completed job is not allowed.

> NOTE: This is going to be changed to return `400 BAD REQUEST` with an informative error message.

Since we can not reuse our failed job (the public API does not support a clone operation) let's [delete it](https://engineering.rescale.com/api-docs/#delete-a-job).

In [ ]:
delete_job_res = requests.delete(f"{api_url}/api/v2/jobs/{job_id}", headers=headers)
delete_job_res.raise_for_status()

print(delete_job_res.status_code)

Now, let's create new job with files attached, submit it and wait till it completes. With the new Job definition, we also specify that we want to run the job using On-Demand Economy (spot market) instances (the `isLowPriority` property).

In [ ]:
job_definition = {
    "name": "Rescale UserApi Reference Tutorial (with inputs)",
    "jobanalyses": [
        {
            "analysis": {"code": "miniconda", "version": "4.8.4"},
            "command": "python calculate_pi.py range.inp",
            "hardware": {"coreType": "granite", "coresPerSlot": 2},
            "inputFiles": [{"id": file1_id}, {"id": file2_id}],
        }
    ],
    "project_id": project_id,
    "isLowPriority": True
}

job_create_res = requests.post(
    f"{api_url}/api/v2/jobs/", headers=headers, json=job_definition
)
job_create_res.raise_for_status()
job_id = job_create_res.json()["id"]

job_create_res = requests.post(
    f"{api_url}/api/v2/jobs/{job_id}/submit/", headers=headers
)
job_create_res.raise_for_status()

poll_for_job_status(job_id, "Completed")


Let's get a list of output files and download them all. Again, Downloading files is a frequent operation, let's encapsulate it in a function.

In [ ]:
from pathlib import Path


def download_file(api_url, headers, file_id, out_dir, file_name):
    response = requests.get(
        f"{api_url}/api/v2/files/{file_id}/contents", headers=headers
    )

    chunk_size = 4096
    with open(Path(out_dir, file_name), "wb") as fd:
        for chunk in response.iter_content(chunk_size):
            fd.write(chunk)


out_dir = "job_outputs"
Path(out_dir).mkdir(exist_ok=True)

files = get_all_result_pages(f"{api_url}/api/v2/jobs/{job_id}/files/", headers)
pprint(files)

for f in files:
    download_file(api_url, headers, f["id"], out_dir, f["name"])


Rescale Files metadata contains the `typeId` property which can have one of the following values.

| id | type                 |
| -- | -------------------- |
| 1  | INPUT_FILE           |
| 2  | TEMPLATE_FILE        |
| 3  | PARAM_FILE           |
| 4  | SCRIPT               |
| 5  | OUTPUT_FILE          |
| 8  | CASE_FILE            |
| 10 | TEMPORARY_FILE       |
| 11 | CHECKPOINT_ARCHIVE   |
| 12 | SNAPSHOT_FILE        |

We can [retrieve metadata](https://engineering.rescale.com/api-docs/#get-metadata-of-a-file) for each file and compare checksums with downloaded files. We fetched this information already when listing job files, here we demonstrate using the `/files` resource.


In [ ]:
import hashlib

for f in files:
    file_metadata_res = requests.get(
        f"{api_url}/api/v2/files/{f['id']}/", headers=headers
    )
    file_metadata_res.raise_for_status()
    remote_hash = file_metadata_res.json()

    hash_sha512 = hashlib.sha512()
    chunk_size = 4096

    with open(Path(out_dir, f["name"]), "rb") as file:
        for chunk in iter(lambda: file.read(chunk_size), b""):
            hash_sha512.update(chunk)

    if hash_sha512.hexdigest() == f["fileChecksums"][0]["fileHash"]:
        print(f"OK\t{f['name']}")
    else:
        print(f"FAIL\t{f['name']}")


Finally let's clean up our job by deleting it together with input files.

In [ ]:
job_delete_res = requests.delete(
    f"{api_url}/api/v2/jobs/{job_id}",
    headers=headers,
    params={"deleteInputFiles": True},
)
job_delete_res.raise_for_status()
print(job_create_res.status_code)


## Optimizing file transfers

### Archiving files before upload

> TODO: Create zip file before submit 

### Parallel file download

Rescale API file download endpoint supports [HTTP Range requests](https://developer.mozilla.org/en-US/docs/Web/HTTP/Range_requests). This allows client code to split download into several chunks and download these in parallel.

Let's start from submitting a job that will generate a 1GB result file.

In [ ]:
TEST_FILE_NAME = "testfile"

job_definition = {
    "name": "Rescale UserApi Reference Tutorial (1GB result file)",
    "jobanalyses": [
        {
            "analysis": {"code": "miniconda", "version": "4.8.4"},
            "command": f"openssl rand -out ${TEST_FILE_NAME} -base64 792917038",
            "hardware": {"coreType": "granite", "coresPerSlot": 2},
        }
    ],
    "project_id": project_id,
}

job_create_res = requests.post(
    f"{api_url}/api/v2/jobs/", headers=headers, json=job_definition
)
job_create_res.raise_for_status()
job_id = job_create_res.json()["id"]

job_create_res = requests.post(
    f"{api_url}/api/v2/jobs/{job_id}/submit/", headers=headers
)
job_create_res.raise_for_status()

poll_for_job_status(job_id, "Completed")


The command of the above Job used `openssl` to create a random file that is now stored in cloud storage. The code below defines several functions that work together to download a file in chucks, each in its own thread. Consult the code comments for explanation.

In [ ]:
import hashlib
import os
import threading
from collections import namedtuple
from pathlib import Path

Chunk = namedtuple("Chunk", ["idx", "start_byte", "end_byte"])
FILE_IO_CHUNK_SIZE = 4096


def _calculate_file_hash(file_path):
    hash_sha512 = hashlib.sha512()

    with open(file_path, "rb") as file:
        for chunk in iter(lambda: file.read(FILE_IO_CHUNK_SIZE), b""):
            hash_sha512.update(chunk)

    return hash_sha512.hexdigest()


def _concatenate_files(output_file, *input_files, delete_chunks=True):
    with open(output_file, "wb") as output:
        for file_name in input_files:
            with open(file_name, "rb") as input_file:
                chunk = input_file.read(FILE_IO_CHUNK_SIZE)
                while chunk:
                    output.write(chunk)
                    chunk = input_file.read(FILE_IO_CHUNK_SIZE)
            if delete_chunks:
                os.remove(file_name)


def _chunk_download(api_url, headers, file_id, dir_path, file_chunk: Chunk):
    headers["Range"] = f"bytes={file_chunk.start_byte}-{file_chunk.end_byte}"
    response = requests.get(
        f"{api_url}/api/v2/files/{file_id}/contents", headers=headers
    )
    response.raise_for_status()

    with open(Path(dir_path, f"{file_id}.{file_chunk.idx}"), "wb") as fd:
        for chunk in response.iter_content(FILE_IO_CHUNK_SIZE):
            fd.write(chunk)


def parallel_download(
    api_url,
    headers,
    file_id,
    dir_path=".",
    file_name=None,
    num_threads=10,
    threshold_mb=50,
):
    # get file size to determine whether splitting makes sense
    response = requests.get(f"{api_url}/api/v2/files/{file_id}/", headers=headers)
    response.raise_for_status()

    decrypted_size = response.json()["decryptedSize"]

    # Check if size is above threshold, if not download in one piece
    num_threads = num_threads if decrypted_size > threshold_mb * 1048576 else 1

    file_hash = response.json()["fileChecksums"][0]["fileHash"]
    file_name = file_name if file_name != None else response.json()["name"]

    chunk_size = int(decrypted_size / num_threads)
    threads = []
    for idx in range(0, num_threads):
        start_byte = idx * chunk_size

        # Make sure the last chunk fetches all remaining bytes
        end_byte = (
            (idx + 1) * chunk_size - 1 if idx != num_threads - 1 else decrypted_size + 1
        )

        t = threading.Thread(
            target=_chunk_download,
            args=(
                api_url,
                headers,
                file_id,
                dir_path,
                Chunk(idx, start_byte, end_byte),
            ),
        )
        t.start()
        threads.append(t)

    for t in threads:
        t.join()

    _concatenate_files(
        file_name, *[f"{file_id}.{idx}" for idx in range(0, num_threads)]
    )

    # Compare file hashes to make sure downloaded file is not corrupted.
    filehash = _calculate_file_hash(file_name)
    if filehash != file_hash:
        raise Exception("File hashes not equal. Corrupted file download.")


# Get test file ID
output_files_res = get_all_result_pages(
    f"{api_url}/api/v2/jobs/{job_id}/files/", headers, params={"search": TEST_FILE_NAME}
)
test_file_id = output_files_res[0]["id"]

# Compare download speed
import time

t1 = time.time()
parallel_download(api_url, headers, test_file_id, num_threads=10)
print(f"Time with 10 threads: {time.time() - t1} s")

t1 = time.time()
parallel_download(api_url, headers, test_file_id, num_threads=1)
print(f"Time with 1 thread: {time.time() - t1} s")


Depending on your connection speed and hardware we should see download time reduction, for example

```
Time with 10 threads: 35.551738023757935 s
Time with 1 thread: 124.59398603439331 s
```



### Parallel upload with archive volumes

> TODO: Split archive into volumes, upload in parallel, merge in command

## Storage devices

> TODO

## Snapshots, Runs, Tasks and DOE Jobs

> TODO

## Clusters

> TODO: /v2/jobs/{job_id}/instances/ /v2/jobs/{job_id}/cluster_statuses/

In [ ]:
cluster_statuses = get_all_result_pages(
    f"{api_url}/api/v2/jobs/{job_id}/cluster_statuses/", headers
)

sorted_statuses = sorted(
    cluster_statuses,
    key=lambda s: datetime.fromisoformat(s["statusDate"].replace("Z", "+00:00")),
)
for s in sorted_statuses:
    print(f"{s['statusDate']}\t{s['status']}")